In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import os
import cv2
import numpy as np
#from xception_gcn_seg import XceptionGCNSegmentation
from resnet_gcn_seg import ResnetGCNSegmentation
#from desnet_gcn_seg import DensenetGCNSegmentation
#from MobileNetDenseASPP import DenseASPP
#from resnext_gcn_seg import ResNeXtGCNSegmentation
#from threeloss_Densenet_DenseASPP_plus import Densenet_DenseASPP_Multiloss
#from threeloss_Densenet_DenseASPP_concate import Densenet_DenseASPP_Multiloss
#from Densenet_DenseASPP import DenseASPP
from resnet_gcn_pab import ResnetGCN_PAB
import matplotlib.pyplot as plt
%matplotlib inline
net = ResnetGCNSegmentation()
model_name = 'snapshot_82_G_model'
net = nn.DataParallel(net).cuda()
checkpoint = torch.load(model_name)
net.load_state_dict(checkpoint)
net.eval()
norm_mean = [0.406, 0.456, 0.485]
norm_std = [0.225, 0.224, 0.229]
norm = {'mean': norm_mean, 'std': norm_std}

num = 1182
Upfile = 'IMG_'+str(num)+'_Seperate/'
os.mkdir(Upfile)
os.mkdir(Upfile+'IMG_'+str(num)+'_sky/')
os.mkdir(Upfile+'IMG_'+str(num)+'_building/')
os.mkdir(Upfile+'IMG_'+str(num)+'_tree/')
os.mkdir(Upfile+'IMG_'+str(num)+'_ground/')
os.mkdir(Upfile+'IMG_'+str(num)+'_person/')
for roots,directions,files in os.walk('IMG_'+str(num)+'/'):
    for imagepath in files:       
        img = cv2.imread(os.path.join(roots,imagepath))
        img_rs = cv2.resize(img,(512,512), interpolation=cv2.INTER_CUBIC)
        input_tensor = img_rs.transpose((2, 0, 1))
        input_tensor = torch.from_numpy(input_tensor).float()
        input_tensor = input_tensor.div(255.0)
        normalize = transforms.Normalize(norm['mean'], norm['std'])
        input_tensor = normalize(input_tensor)
        input_tensor = torch.unsqueeze(input_tensor, 0)
        input_tensor = Variable(input_tensor, volatile=True)
        outputs = net(input_tensor.cuda())
        img_mask = outputs['output'].cpu().data.numpy()
        mask = np.squeeze(img_mask).astype('uint8')
        mask = cv2.resize(mask,(img.shape[1],img.shape[0]),interpolation=cv2.INTER_NEAREST)
        sky =np.zeros((mask.shape[0],mask.shape[1],4))
        building =np.zeros((mask.shape[0],mask.shape[1],4))
        tree =np.zeros((mask.shape[0],mask.shape[1],4))
        ground =np.zeros((mask.shape[0],mask.shape[1],4))
        person =np.zeros((mask.shape[0],mask.shape[1],4))
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i][j] ==0:
                    sky[i][j] = np.append(img[i][j],255)
                elif mask[i][j] ==1:
                    building[i][j] = np.append(img[i][j],255)
                elif mask[i][j] ==2:
                    tree[i][j] = np.append(img[i][j],255)
                elif mask[i][j] ==3:
                    ground[i][j] = np.append(img[i][j],255)
                elif mask[i][j] ==4:
                    person[i][j] = np.append(img[i][j],255)
        cv2.imwrite(Upfile+'IMG_'+str(num)+'_sky/'+imagepath[:-4]+'_sky.png',sky)
        cv2.imwrite(Upfile+'IMG_'+str(num)+'_building/'+imagepath[:-4]+'_building.png',building)
        cv2.imwrite(Upfile+'IMG_'+str(num)+'_tree/'+imagepath[:-4]+'_tree.png',tree)
        cv2.imwrite(Upfile+'IMG_'+str(num)+'_person/'+imagepath[:-4]+'_person.png',person)
        cv2.imwrite(Upfile+'IMG_'+str(num)+'_ground/'+imagepath[:-4]+'_ground.png',ground)

resnet_gcn_seg.py:380: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax_output = F.softmax(out)
